In [1]:
conda install -c conda-forge koalas


Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


### create spark session via bq connector

In [2]:
# # %load sparkme
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.getOrCreate()
# scala_minor_version = str(spark.sparkContext._jvm.scala.util.Properties.versionString().replace("version ","").split('.')[1])
# spark = SparkSession.builder.config("spark.jars.packages", "gs://spark-lib/bigquery/spark-bigquery-with-dependencies_2.11-0.29.0.jar") \
#                                     .enableHiveSupport() \
#                                     .getOrCreate()

24/04/11 12:45:27 WARN Utils: Your hostname, mad-scientist resolves to a loopback address: 127.0.1.1; using 192.168.0.161 instead (on interface wlp0s20f3)
24/04/11 12:45:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/11 12:45:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/11 12:45:29 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### setup gcp bq client connector

In [3]:
! pip3 install google-cloud-bigquery


In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
credentials = service_account.Credentials.from_service_account_file('trauco_bq_spark_credentials.json')

project_id = 'data-nasa'
client = bigquery.Client(credentials= credentials,project=project_id)

### create client query

In [4]:
query = client.query("""
   SELECT * FROM `bigquery-public-data.fda_food.food_events` LIMIT 1000
 """)

results = query.result()
for row in results:
	print(row)

Row(('163487', '"ASPERGERS DISORDER", ATTENTION DEFICIT/HYPERACTIVITY DISORDER, AUTISM, BIRTH TRAUMA, DRUG EXPOSURE DURING PREGNANCY, SKULL MALFORMATION', 'Hospitalization, Other Serious or Important Medical Event', 'FLINTSTONES COMPLETE (MULTIVITAMINS + MINERALS) CHEWABLE TABLET', '54', 'SUSPECT', 'Vit/Min/Prot/Unconv Diet(Human/Animal)', datetime.date(2013, 3, 15), datetime.date(2003, 8, 13), 'Male', None, None), {'report_number': 0, 'reactions': 1, 'outcomes': 2, 'products_brand_name': 3, 'products_industry_code': 4, 'products_role': 5, 'products_industry_name': 6, 'date_created': 7, 'date_started': 8, 'consumer_gender': 9, 'consumer_age': 10, 'consumer_age_unit': 11})
Row(('140976', '"BARRETTS OESOPHAGUS"', 'Other Serious or Important Medical Event', 'CITRACAL PLUS (CALCIUM CARBONATE + VITAMIN D) FILM-COATED TABLET', '54', 'SUSPECT', 'Vit/Min/Prot/Unconv Diet(Human/Animal)', datetime.date(2011, 6, 22), None, 'Male', None, None), {'report_number': 0, 'reactions': 1, 'outcomes': 2, '

### read data into df



In [5]:
! pip3 install db-dtypes

In [6]:
import db_dtypes

In [16]:
! pip3 install 'google-cloud-bigquery[pandas]'

In [7]:

import db_dtypes
query = client.query("""
   SELECT * FROM `bigquery-public-data.fda_food.food_events` LIMIT 1000
 """)

result_df = query.to_dataframe()

In [8]:
result_df.head()


,report_number,reactions,outcomes,products_brand_name,products_industry_code,products_role,products_industry_name,date_created,date_started,consumer_gender,consumer_age,consumer_age_unit
0,163487,"""ASPERGERS DISORDER"", ATTENTION DEFICIT/HYPERA...","Hospitalization, Other Serious or Important Me...",FLINTSTONES COMPLETE (MULTIVITAMINS + MINERALS...,54,SUSPECT,Vit/Min/Prot/Unconv Diet(Human/Animal),2013-03-15,2003-08-13,Male,NaN,None
1,140976,"""BARRETTS OESOPHAGUS""",Other Serious or Important Medical Event,CITRACAL PLUS (CALCIUM CARBONATE + VITAMIN D) ...,54,SUSPECT,Vit/Min/Prot/Unconv Diet(Human/Animal),2011-06-22,NaT,Male,NaN,None
2,140976,"""BARRETTS OESOPHAGUS""",Other Serious or Important Medical Event,VITAMIN C,54,CONCOMITANT,Vit/Min/Prot/Unconv Diet(Human/Animal),2011-06-22,NaT,Male,NaN,None
3,109790,"""BARRETTS OESOPHAGUS"", DYSPHAGIA, DYSPNOEA, HY...",Visited Emergency Room,EGG BEATERS EGG SUBSTITUTE,15,SUSPECT,Egg/Egg Prod,2008-10-14,NaT,Male,NaN,None
4,131807,"""BASEDOWS DISEASE""",Other Serious or Important Medical Event,ONE A DAY WOMEN'S,54,SUSPECT,Vit/Min/Prot/Unconv Diet(Human/Animal),2010-10-05,NaT,Female,NaN,None


### pyarrow

In [9]:

! pip3 install pyarrow



In [10]:
query = client.query("""
   SELECT * FROM `bigquery-public-data.fda_food.food_events` LIMIT 1000
 """)

result_df = query.to_dataframe()

## you could do any manipulation to the dataframe you want here

table_id = 'your_dataset_name.new_table_name'

## optional: you can pass in a job_config with a schema if you want to define
## a specific schema
# job_config = bigquery.LoadJobConfig(schema=[
#     bigquery.SchemaField("field_name", "STRING"),
# ])

load_job = client.load_table_from_dataframe(
    df, table_id,
# job_config=job_config
)  # this will execute the load job

result = load_job.result()
result.done() # prints True if done

NameError: name 'df' is not defined

### spark vs scale connector


In [12]:
! export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64

In [14]:
! echo $JAVA_HOME

In [13]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .config("spark.jars.packages", "gs://spark-lib/bigquery/spark-3.4-bigquery-0.37.0.jar") \
  .getOrCreate()
df = spark.read.format("bigquery") \
  .load("dataset.table")


24/04/11 14:04:51 WARN Utils: Your hostname, mad-scientist resolves to a loopback address: 127.0.1.1; using 192.168.0.161 instead (on interface wlp0s20f3)
24/04/11 14:04:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/trauco/miniconda3/envs/tensorflow/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Exception in thread "main" java.lang.IllegalArgumentException: requirement failed: Provided Maven Coordinates must be in the form 'groupId:artifactId:version'. The coordinate provided is: gs://spark-lib/bigquery/spark-3.4-bigquery-0.37.0.jar
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.deploy.SparkSubmitUtils$.$anonfun$extractMavenCoordinates$1(SparkSubmit.scala:1212)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.map(TraversableLike.scala:286)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:279)
	at scala.collection.mutable.ArrayOps$ofRef.map(ArrayOps.scala:198)
	at org.apache.spark.deploy.SparkSubmitUtils$.extractMavenCoordinates(SparkSubmit.scala:1210)


RuntimeError: Java gateway process exited before sending its port number

In [17]:
! export PYSPARK_SUBMIT_ARGS="--master local[2] pyspark-shell"


In [18]:
def start_or_create_spark():
    from pyspark.sql import SparkSession
    spark = (SparkSession
             .builder
             .appName("Processamento de Dados de Gasolina no Brasil")
             .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-with-dependencies_2.12-0.23.2.jar')
             .getOrCreate()
             )
    return spark

In [19]:
start_or_create_spark()

24/04/11 14:14:27 WARN Utils: Your hostname, mad-scientist resolves to a loopback address: 127.0.1.1; using 192.168.0.161 instead (on interface wlp0s20f3)
24/04/11 14:14:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/04/11 14:14:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Exception in thread "main" org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.spark.util.DependencyUtils$.resolveGlobPath(DependencyUtils.scal

RuntimeError: Java gateway process exited before sending its port number